In [ ]:
import os

fig_out_dir = "figures/case_study_visualizations"
os.makedirs(fig_out_dir, exist_ok=True)

In [ ]:
from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection/"
CSM = CarlaScenesManager(data_dir=data_root, split="train")

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes
from PIL import Image

# loop over scene to make images
for idx_scene in range(4):
    CDM = CSM.get_scene_dataset_by_index(idx_scene)
    scene_dir = os.path.join(fig_out_dir, f"scene{idx_scene}")
    os.makedirs(scene_dir, exist_ok=True)

    # loop over agents and frames
    idx_frames = [20, 40, 60]
    frame_agent_tuples = [
        (idx_frame, ID_agent) for ID_agent in range(4) for idx_frame in idx_frames
    ]
    for idx_frame, ID_agent in frame_agent_tuples:
        frames = CDM.get_frames(sensor="camera-0", agent=ID_agent)

        # get boxes near the ego
        objs = CDM.get_objects_global(
            frame=frames[idx_frame], ignore_static_agents=True
        )
        if ID_agent == 0:
            ego = CDM.get_agent(frame=frames[idx_frame], agent=0)
            objs_filter = [obj for obj in objs if 2 < obj.distance(ego) < 15]
        else:
            objs_filter = objs

        # get image
        frame = frames[idx_frame]
        img_ego = CDM.get_image(
            frame=frames[idx_frame], sensor="camera-0", agent=ID_agent
        )
        base_filename = os.path.join(
            scene_dir, "agent{}_frame{}_image{}".format(ID_agent, frame, "{}")
        )

        # ----------------------------------
        # Image without boxes
        # ----------------------------------
        im = Image.fromarray(img_ego.data)
        im.save(base_filename.format("_noboxes.pdf"))
        im.save(base_filename.format("_noboxes.png"))

        # ----------------------------------
        # Image with boxes
        # ----------------------------------

        # show image
        img_cv_out = show_image_with_boxes(
            img_ego, objs_filter, inline=True, return_image=True
        )

        # save image
        im = Image.fromarray(img_cv_out)
        im.save(base_filename.format(".pdf"))
        im.save(base_filename.format(".png"))